In [9]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")

In [10]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'], axis=1)

In [11]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

Noisy variables are removed from features.

In [12]:
# APPLICATION_TYPE Data Processing
application_df = application_df.drop(columns=['APPLICATION_TYPE'], axis=1)

In [13]:
# CLASSIFICATION Data Processing

# Look at CLASSIFICATION value counts for binning
class_counts = application_df.CLASSIFICATION.value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(class_counts[class_counts < 1883].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [14]:
# Categorical Processing

# Generate our categorical variable lists
application_cat = ["AFFILIATION","CLASSIFICATION","USE_CASE","ORGANIZATION","INCOME_AMT","SPECIAL_CONSIDERATIONS"]

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True).drop(columns=application_cat, axis=1)

In [8]:
application_df.nunique()

STATUS                             2
ASK_AMT                         8747
IS_SUCCESSFUL                      2
APPLICATION_TYPE_Other             2
APPLICATION_TYPE_T10               2
APPLICATION_TYPE_T19               2
APPLICATION_TYPE_T3                2
APPLICATION_TYPE_T4                2
APPLICATION_TYPE_T5                2
APPLICATION_TYPE_T6                2
APPLICATION_TYPE_T7                2
APPLICATION_TYPE_T8                2
AFFILIATION_CompanySponsored       2
AFFILIATION_Family/Parent          2
AFFILIATION_Independent            2
AFFILIATION_National               2
AFFILIATION_Other                  2
AFFILIATION_Regional               2
CLASSIFICATION_C1000               2
CLASSIFICATION_C1200               2
CLASSIFICATION_C2000               2
CLASSIFICATION_C2100               2
CLASSIFICATION_C3000               2
CLASSIFICATION_Other               2
USE_CASE_CommunityServ             2
USE_CASE_Heathcare                 2
USE_CASE_Other                     2
U

In [15]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop("IS_SUCCESSFUL",axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,stratify=y)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Additional hidden layers + Additional neurons per hidden layer

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 40
nodes_hidden_layer3 = 40
nodes_hidden_layer4 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

2022-05-02 21:50:26.387823: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-02 21:50:26.390983: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                2800      
_________________________________________________________________
dense_1 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_2 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_3 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 41        
Total params: 9,361
Trainable params: 9,361
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints2/", exist_ok=True)
checkpoint_path = "checkpoints2/weights.{epoch:02d}hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(np.asarray(X_train_scaled),np.asarray(y_train),epochs=100,callbacks=[cp_callback])

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 5s 205us/sample - loss: 0.5930 - accuracy: 0.6976
Epoch 2/100
25724/25724 [==============================] - 3s 115us/sample - loss: 0.5812 - accuracy: 0.7049
Epoch 3/100
25724/25724 [==============================] - 3s 110us/sample - loss: 0.5784 - accuracy: 0.7074
Epoch 4/100
25724/25724 [==============================] - 3s 109us/sample - loss: 0.5774 - accuracy: 0.7077
Epoch 5/100
25504/25724 [============================>.] - ETA: 0s - loss: 0.5772 - accuracy: 0.7091
Epoch 00005: saving model to checkpoints2/weights.05hdf5
25724/25724 [==============================] - 3s 120us/sample - loss: 0.5771 - accuracy: 0.7089
Epoch 6/100
25724/25724 [==============================] - 4s 136us/sample - loss: 0.5758 - accuracy: 0.7093
Epoch 7/100
25724/25724 [==============================] - 4s 164us/sample - loss: 0.5752 - accuracy: 0.7089
Epoch 8/100
25724/25724 [==============================] - 6s 219us/

25724/25724 [==============================] - 22s 858us/sample - loss: 0.5674 - accuracy: 0.7150
Epoch 60/100
25664/25724 [============================>.] - ETA: 0s - loss: 0.5675 - accuracy: 0.7157
Epoch 00060: saving model to checkpoints2/weights.60hdf5
25724/25724 [==============================] - 19s 743us/sample - loss: 0.5674 - accuracy: 0.7158
Epoch 61/100
25724/25724 [==============================] - 10s 402us/sample - loss: 0.5673 - accuracy: 0.7158
Epoch 62/100
25724/25724 [==============================] - 17s 651us/sample - loss: 0.5672 - accuracy: 0.7152
Epoch 63/100
25724/25724 [==============================] - 18s 690us/sample - loss: 0.5674 - accuracy: 0.7162
Epoch 64/100
25724/25724 [==============================] - 18s 706us/sample - loss: 0.5673 - accuracy: 0.7156
Epoch 65/100
25280/25724 [============================>.] - ETA: 0s - loss: 0.5674 - accuracy: 0.7152
Epoch 00065: saving model to checkpoints2/weights.65hdf5
25724/25724 [=============================

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(np.asarray(X_test_scaled),np.asarray(y_test),verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5606 - accuracy: 0.7100
Loss: 0.5834909761890378, Accuracy: 0.7099708318710327


In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 40
nodes_hidden_layer3 = 40
nodes_hidden_layer4 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer3, activation="tanh"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer4, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 80)                2800      
_________________________________________________________________
dense_6 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_7 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_8 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 41        
Total params: 9,361
Trainable params: 9,361
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints2/", exist_ok=True)
checkpoint_path = "checkpoints2/weights.{epoch:02d}hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(np.asarray(X_train_scaled),np.asarray(y_train),epochs=100,callbacks=[cp_callback])

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 5s 202us/sample - loss: 0.5928 - accuracy: 0.6945
Epoch 2/100
25724/25724 [==============================] - 4s 137us/sample - loss: 0.5817 - accuracy: 0.7063
Epoch 3/100
25724/25724 [==============================] - 3s 128us/sample - loss: 0.5798 - accuracy: 0.7072
Epoch 4/100
25724/25724 [==============================] - 3s 134us/sample - loss: 0.5787 - accuracy: 0.7064
Epoch 5/100
25536/25724 [============================>.] - ETA: 0s - loss: 0.5784 - accuracy: 0.7083
Epoch 00005: saving model to checkpoints2/weights.05hdf5
25724/25724 [==============================] - 4s 138us/sample - loss: 0.5784 - accuracy: 0.7082
Epoch 6/100
25724/25724 [==============================] - 4s 144us/sample - loss: 0.5777 - accuracy: 0.7088
Epoch 7/100
25724/25724 [==============================] - 4s 140us/sample - loss: 0.5769 - accuracy: 0.7091
Epoch 8/100
25724/25724 [==============================] - 3s 122us/

25724/25724 [==============================] - 4s 144us/sample - loss: 0.5675 - accuracy: 0.7152
Epoch 60/100
25664/25724 [============================>.] - ETA: 0s - loss: 0.5674 - accuracy: 0.7144
Epoch 00060: saving model to checkpoints2/weights.60hdf5
25724/25724 [==============================] - 4s 146us/sample - loss: 0.5674 - accuracy: 0.7145
Epoch 61/100
25724/25724 [==============================] - 8s 299us/sample - loss: 0.5671 - accuracy: 0.7140
Epoch 62/100
25724/25724 [==============================] - 11s 422us/sample - loss: 0.5670 - accuracy: 0.7149
Epoch 63/100
25724/25724 [==============================] - 10s 389us/sample - loss: 0.5674 - accuracy: 0.7154
Epoch 64/100
25724/25724 [==============================] - 6s 221us/sample - loss: 0.5673 - accuracy: 0.7140
Epoch 65/100
25696/25724 [============================>.] - ETA: 0s - loss: 0.5671 - accuracy: 0.7140
Epoch 00065: saving model to checkpoints2/weights.65hdf5
25724/25724 [==============================] -

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(np.asarray(X_test_scaled),np.asarray(y_test),verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5425 - accuracy: 0.7092
Loss: 0.5793476585744074, Accuracy: 0.7091545462608337


In [25]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")